In [1]:
import sys
sys.path.append("/Users/karinabalagazova/Desktop/cvut/5.semestr/scientificProject/notebooks/..")

# from functions.experiment import experiment_iteration
from functions.rnaseq_data_generator import rna_seq_generator, NormalDistributionParameters
from utils.enums import Distribution
%load_ext autoreload
%autoreload 2
import numpy as np

from functions.saving_data import get_empty_auc_dataset
import time
from multiprocessing import Pool

In [2]:
# Prepare to store data into array
STA_range = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000]
data = get_empty_auc_dataset(STA_range, 2)

In [3]:
# Save data
np.savez('auc_results_origin.npz', auc_data=data)
saved_origin = np.load('auc_results_origin.npz', allow_pickle=True)
saved_origin = saved_origin['auc_data'].item()

np.savez('auc_results_expected.npz', auc_data=data)
saved_expected = np.load('auc_results_expected.npz', allow_pickle=True)
saved_expected = saved_expected['auc_data'].item()

np.savez('auc_results_filtered.npz', auc_data=data)
saved_filtered = np.load('auc_results_filtered.npz', allow_pickle=True)
saved_filtered = saved_filtered['auc_data'].item()

In [4]:
import statsmodels.api as sm

def work_bitch(df):
#     if distribution == "Poisson":
#         mp = sm.formula.glm("y ~ x1 + x2 + x3", family=sm.families.Poisson(), data=df).fit()
#     elif distribution == "Negative binomial":
    mp = sm.formula.glm("y ~ x1 + x2 + x3", family=sm.families.NegativeBinomial(), data=df).fit()
    return np.array([mp.params.x1, mp.params.x2, mp.params.x3])
#     else:
#         print("Unknown distribution")
#         return

def compensation(counts, IS, t, n_CR, distribution, pool, filename):
    """
    IS compensation.

    :param counts: Counts to be compensated.
    :param IS: Array of immunosuppression intake (0 or 1).
    :param t: Number of transcripts.
    :param distribution: "Poisson" or "Negative binomial".
    :param n_CR: Number of samples in CR class. First n_CR elements of counts must be of CR class.
    """
    # leave only STA
    STA_patients = counts.columns.str.startswith('STA')
    STA_counts = counts.loc[:, STA_patients].to_numpy()
    STA_IS = IS.loc[STA_patients]

    # try to learn the beta1 parameters from STA class
    # TODO: save learned_beta and use it to speed up code
    learned_beta = np.zeros((3, t))
    values = list()
    for ind in range(t):
        values.append(pd.DataFrame({"x1": STA_IS['IS1'].values.tolist(),
                           "x2": STA_IS['IS2'].values.tolist(),
                           "x3": STA_IS['IS3'].values.tolist(),
                           "y": STA_counts[ind]}))

#     with Pool() as pool:
    vs = pool.map(work_bitch, values)

    for ind in range(t):
        learned_beta[:, ind] = vs[ind]
#     return
#         df = pd.DataFrame({"x1": STA_IS['IS1'].values.tolist(),
#                            "x2": STA_IS['IS2'].values.tolist(),
#                            "x3": STA_IS['IS3'].values.tolist(),
#                            "y": STA_counts[ind]})
#         if distribution == "Poisson":
#             mp = sm.formula.glm("y ~ x1 + x2 + x3", family=sm.families.Poisson(), data=df).fit()
#         elif distribution == "Negative binomial":
#             mp = sm.formula.glm("y ~ x1 + x2 + x3", family=sm.families.NegativeBinomial(), data=df).fit()
#         else:
#             print("Unknown distribution")
#             return
#         learned_beta[:, ind] = np.array([mp.params.x1, mp.params.x2, mp.params.x3])


    np.savez('{filename}_learnedbeta.npz'.format(filename=filename), beta=learned_beta)
    
    # leave only CR and OT classes for classification
    OT_CR_counts = counts.loc[:, ~STA_patients].to_numpy(dtype=float)
    OT_CR_IS = IS.loc[~STA_patients].values

    # filter out IS from CR
    OT_CR_counts[:, :n_CR] = OT_CR_counts[:, :n_CR] / np.exp(np.dot(OT_CR_IS[:n_CR], learned_beta).transpose())
    OT_CR_counts = OT_CR_counts.transpose()
    return OT_CR_counts, OT_CR_IS, learned_beta

In [5]:
def experiment_classification(n_transcripts, n_IS_effect, IS_effect, n_class_effect, class_effect_mean, STA_range, pool, filename):
    n_repeats = 10
    n_genSTA = 7000 # pocet nagenerovanych STA
    n_genCR = 100
    n_genOT = 100
    _, _, _, gen_origin, gen_expected, gen_IS = rna_seq_generator(
        n_transcripts=n_transcripts,
        distribution=Distribution.NEGATIVE_BINOMIAL, #"Negative binomial",
        n_STA=n_genSTA, n_CR=n_genCR, n_OT=n_genOT,
        n_IS_effect=n_IS_effect, IS_effect=(IS_effect, IS_effect+0.01),  # IS effect
        is_class_effect=n_class_effect > 0, n_class_effect=n_class_effect, class_effect=NormalDistributionParameters(class_effect_mean, 0.1)
    )
    mean_auc_results_origin, mean_auc_results_expected, mean_auc_results_filtered = \
        experiment_iteration(STA_range, n_repeats,
                             gen_origin, gen_expected, gen_IS,
                             n_genSTA, n_genCR, n_genOT, "Negative binomial", pool, filename) #"Negative binomial")
    return mean_auc_results_origin, mean_auc_results_expected, mean_auc_results_filtered

In [7]:
%%time

transcripts = [1000]#[100, 1000] #, 10000]
class_effect_n_transcripts = [2]#[0, 2, 5, 10]
class_effect = [0.2] #, 0.3, 0.4]  # mean of normal distribution

IS_effect_n_transcripts = [2]#, 5, 10]
IS_effect = [0.3]#, 0.5, 0.9]  # coefficients beta1..beta3

# transcripts = [10000]
# class_effect_n_transcripts = [0, 5, 10]
# class_effect = [0.3, 0.4]  # mean of normal distribution

# IS_effect_n_transcripts = [5, 10]
# IS_effect = [0.5, 0.9]  # coefficients beta1..beta3
with Pool() as pool:
    for t in transcripts:
        for n_IS in IS_effect_n_transcripts:
            print('new IS_effect_n_transcripts: ', n_IS)
            for i in IS_effect:
                print('new IS_effect: ', i)
                for n_class in class_effect_n_transcripts:
                    if n_class == 0:
                        c = 0.2
                        filename = 'generateddata04_08/{t}_{n_IS}_{i}_{n_class}'.format(t=t, n_IS=n_IS, i=i, n_class=n_class)
                        mean_o, mean_e, mean_f = experiment_classification(t, n_IS, i, n_class, c, STA_range, pool, filename)
                        saved_origin[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                        saved_origin[t][n_IS][i][n_class][c]['auc'] = mean_o

                        saved_expected[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                        saved_expected[t][n_IS][i][n_class][c]['auc'] = mean_e

                        saved_filtered[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                        saved_filtered[t][n_IS][i][n_class][c]['auc'] = mean_f
                    else:
                        for c in class_effect:
                            STA_range = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000]
                            
                            filename = 'generateddata04_08/{t}_{n_IS}_{i}_{n_class}_{c}'.format(t=t, n_IS=n_IS, i=i, n_class=n_class, c=c)

                            mean_o, mean_e, mean_f = experiment_classification(t, n_IS, i, n_class, c, STA_range, pool, filename)
                            saved_origin[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                            saved_origin[t][n_IS][i][n_class][c]['auc'] = mean_o

                            saved_expected[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                            saved_expected[t][n_IS][i][n_class][c]['auc'] = mean_e

                            saved_filtered[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                            saved_filtered[t][n_IS][i][n_class][c]['auc'] = mean_f

                np.savez('auc_results_origin.npz', auc_data=saved_origin)
                np.savez('auc_results_expected.npz', auc_data=saved_expected)
                np.savez('auc_results_filtered.npz', auc_data=saved_filtered)

new IS_effect_n_transcripts:  2
new IS_effect:  0.3


NameError: name 'experiment_iteration' is not defined

In [ ]:
# Save data
np.savez('auc_results_origin5-poisson.npz', auc_data=saved_origin)
np.savez('auc_results_expected5-poisson.npz', auc_data=saved_expected)
np.savez('auc_results_filtered5-poisson.npz', auc_data=saved_filtered)

In [ ]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

In [20]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

[1, 4, 9]
